In [2]:
import pandas as pd
import requests
import numpy as np
import json
import datetime
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', None)

def run_query(query):  # A simple function to use requests.post to make the API call.
    headers = {'X-API-KEY': 'BQYCaXaMZlqZrPCSQVsiJrKtxKRVcSe4'}
    request = requests.post('https://graphql.bitquery.io/', json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        print(request.reason)
        raise Exception('Query failed and return code is {}.{}'.format(request.status_code, query))

In [3]:
query_date = '"2021-12-16", "{}"'.format(str(datetime.date.today()))

In [4]:
signature_list = str(["5UHdyopipQSkskE1bgZ4Vuz1KK6dNfXBNAViLsirotCjpefoFNAgsM8iErdBKFjrPUPNNwMbHy5V4it9Zj8jpAXX"]*100)[1:-1]

In [5]:
signature_list = signature_list.replace("\'", "\"")

In [6]:
query = """query MyQuery {
  solana {
    instructions(
      programId: {is: "VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp"}
      signature: {in: [""" + signature_list + """]}
      date: {between: ["2022-03-01", "2022-03-25"]}
      success: {is: true}
    ) {
      log {
        consumed
        instruction
        logs
      }
      transaction {
        signature
        success
        feePayer
      }
      data {
        base58
        hex
      }
      block {
        timestamp {
          iso8601
        }
      }
    }
  }
}

"""

In [68]:
result = run_query(query)
# convert GraphQL json to pandas dataframe
df = pd.json_normalize(result['data']['solana']['instructions'])

In [69]:
df = df.rename(columns={"block.timestamp.iso8601": "date", "currency.symbol": "symbol", 
                        "transaction.signer":"signer", "currency.address":"cash",
                        "receiver.address":"receiver", "sender.address":"sender", "transaction.signature": "tx_id"})

In [70]:
df.groupby("log.instruction")["data.hex"].count()

log.instruction
Withdraw    1
Name: data.hex, dtype: int64

In [71]:
df.loc[df["log.instruction"]=="Withdraw"]

,log.consumed,log.instruction,log.logs,tx_id,transaction.success,transaction.feePayer,data.base58,data.hex
0,135080,Withdraw,Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp invoke [1];Program log: Instruction: Withdraw;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp consumed 135080 of 200000 compute units;Program VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp success,5UHdyopipQSkskE1bgZ4Vuz1KK6dNfXBNAViLsirotCjpefoFNAgsM8iErdBKFjrPUPNNwMbHy5V4it9Zj8jpAXX,True,2E3Kn91xgPwux9uxRgjJsKKpcZuQA23wiw2BeAWv6WaQ,PcB3tF1KHa1mxTi5gB1xbH,000001480dc09d0022a16d949c4612b7


In [72]:
df.loc[df["log.instruction"]=="Deposit"]

,log.consumed,log.instruction,log.logs,tx_id,transaction.success,transaction.feePayer,data.base58,data.hex
